# LangChaing version: Questions from YT transcript video (MORE THAN 1 DOCUMENT)

In [1]:
import os
import openai
import sys
sys.path.append('../..')
from openai import AzureOpenAI
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv()) # read local .env file


AZURE_OPENAI_ENDPOINT_GPT = os.getenv('AZURE_OPENAI_ENDPOINT_GPT')
AZURE_OPENAI_ENDPOINT_EMBEDDING = os.getenv('AZURE_OPENAI_ENDPOINT_EMBEDDING')
AZURE_OPENAI_API_KEY = os.getenv('AZURE_OPENAI_API_KEY')
AZURE_OPENAI_API_VERSION = os.getenv('AZURE_OPENAI_API_VERSION')
OPENAI_MODEL_NAME = os.getenv('OPENAI_MODEL_NAME')
OPENAI_GPT4_TURBO_ENGINE = os.getenv('OPENAI_GPT4_TURBO_ENGINE')

### 1)Document loading

In [2]:
import os
import glob

###################
from langchain_community.document_loaders import TextLoader

# Specify the folder path
folder_path = 'mauritania_dataset/'

# Use glob to get all text files in the folder
file_paths = glob.glob(os.path.join(folder_path, '*.txt'))

docs = []
# Iterate over each file and read its content
for file_path in file_paths:
    loader =TextLoader(file_path)
    docs.extend(loader.load())

In [3]:
docs

[Document(metadata={'source': 'mauritania_dataset/Clavero - Mauritania.txt'}, page_content="en el año 1982 mercedes sacó a la venta\nel w 201 de la clase de un coche\nprecioso y robusto que tuvo muchísimo\néxito en europa este modelo se produjo\nhasta el año 1993 fabricándose un total\nde un millón 874 mil 668 mirado en el\ntiempo este modelo fue cogiendo polvo y\ndesapareciendo en las calles europeas\npero no del mundo y es que la mayoría de\nestos mercedes acabaron en un\ndesconocido país africano mauritania un\nmercedes dos mercedes\n3 10 11 12 13 14 15 6 20 hay muchísimas\npartes que todo lo que se ve en el\naparcamiento prácticamente son mercedes\na un mercedes y aparca a otros\ne\n[Música]\nes una locura porque en este país puedes\nb-20 mercedes apelotonados uno detrás de\notro todo lo que ves a tu alrededor es\ncarrocería mercedes mercedes de los años\n80 90 de hecho me apartan porque quieren\naparcar convertidor o mauritania es uno\nde los países más pobres del mundo como\npued

### 2)Creating the model

In [4]:
from langchain_openai import AzureChatOpenAI
# To control the randomness and creativity of the generated
# text by an LLM, use temperature = 0.0
llm_model = 'gpt-4o'
chat = AzureChatOpenAI(temperature=0.0, 
                       azure_deployment=llm_model, 
                       api_version='2024-02-15-preview',
                       azure_endpoint = AZURE_OPENAI_ENDPOINT_GPT)


Creating a template for the prompt, to repeatly use it

In [11]:
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [45]:
# We define the schema we want for the outputs, then we introduce them into a list. 

video_name_schema = ResponseSchema(name="video_name",
                                    description="Cual es el nombre del video\
                                                de donde se esta extrayendo la información\
                                                en el caso de no estar presente la respuesta debe ser\
                                                'desconocido'")

youtuber_name_schema = ResponseSchema(name="youtuber_name",
                                    description="Cual es el nombre de la persona que esta haciendo el video\
                                                en el caso de no estar presente la respuesta debe ser\
                                                'desconocido'")

video_interesting_facts_schema = ResponseSchema(name="datos_interesantes",
                                      description="Debe ser una lista extensa y detallada con todos los datos interesantes\
                                                   que se puedan extraer del script del video. Cuantos más datos interesantes se extraigan mejor")

country_schema = ResponseSchema(name="pais",
                                    description="Cual es el nombre del pais que se visita en el video\
                                            en el caso de no estar presente la respuesta debe ser\
                                                'desconocido")

response_schemas = [video_name_schema, 
                    youtuber_name_schema,
                    video_interesting_facts_schema,
                    country_schema]


output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()


In [33]:
docs[1].metadata['source'][19:-4]

'Clavero - pais que es 99% desierto'

In [48]:
template_prompt = """Extrae todos los datos interesantes que se encuentren \
en el  texto delimitado por triple comas \
, el texto trata sobre el script de un video de YouTube llamado {youtube_video_name}\
el output debe ser un diccionario key valor \
text: ```{text}```

{format_instructions}
"""
answers = []

prompt = ChatPromptTemplate.from_template(template=template_prompt)

for i in range(len(docs)):
    messages = prompt.format_messages(text=docs[i].page_content, 
                                    youtube_video_name = docs[i].metadata['source'][19:-4],
                                    format_instructions=format_instructions)

    #We send the prompt to the LLM
    response = chat(messages)

    # We parse the output into a dict 
    output_dict = output_parser.parse(response.content)
    answers.append(output_dict)

In [61]:
type(answers[0]['datos_interesantes'])

list

In [20]:
#! pip install python-docx

Below is the code for saving each value extracted from the YT transcrip.

In [73]:
from docx import Document

# Create a new Document
doc = Document()

# Add a title
doc.add_heading('Mauritania', level=1)

for i in range(len(answers)):
    # Add a paragraph
    doc.add_paragraph('Resumen del video ' + answers[i]['video_name'] )

    for j in list(answers[i].keys()):
    # Add a bullet point list
        if type(answers[i][j]) != list:
            
            if j == 'video_name':
                # Añadir un título de nivel 2
                doc.add_heading(answers[i][j], level=2)

            else:   
                doc.add_paragraph(answers[i][j], style='ListBullet')

        else:
            for z in answers[i][j]:
              doc.add_paragraph(z, style= 'Normal')  

# Save the document
doc.save('Mauritania_v2.docx')

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/docx/styles/styles.py:130: UserWarning: style lookup by style_id is deprecated. Use style name as key instead.
  return self._get_style_id_from_style(self[style_name], style_type)
